In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import KFold
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [4]:
data = pd.read_csv('dulieuthunho.csv')
data['Ngày'] = pd.to_datetime(data['Ngày'])

In [5]:
features = ['Mở cửa', 'Đóng cửa', 'Cao nhất', 'Thấp nhất', 'Trung bình', 'GD khớp lệnh KL']
data = data[['Ngày', 'Mã CK'] + features]
data = data.sort_values(by=['Mã CK', 'Ngày'])

In [6]:
def prepare_data(data, features):
    data_scaled = data.copy()#copy ra một cái dữ liệu để ko làm thay đổi dữ liệu gốc
    scaler = MinMaxScaler(feature_range=(0, 1)) # min max Scaler với gtri 01
    data_scaled[features] = scaler.fit_transform(data[features])# áp dụng cho features
    return data_scaled, scaler# trả về kết vào data_sacled
# chuẩn hóa dữ liệu

In [7]:
def create_sequences(data, seq_length, feature_col): # tạo trình tự cho dữ liệu gồm có dữ liệu, độ dài chuỗi dữ liệu, cột dữ đoán muốn dùng
    X, Y = [], [] # tạo 2 danh sách rỗng X chứa các chuỗi dữ liệu đầu vào cho mô hình (các chuỗi dữ liệu dài seq_length, Y: Sẽ chứa các giá trị mục tiêu tương ứng (giá trị tại thời điểm tiếp theo sau chuỗi X)
    for i in range(len(data) - seq_length): #Vòng lặp này sẽ lặp qua từng chỉ số i từ 0 đến len(data) - seq_length - 1, vào đảm vào không có độ dài lớn hơn độ dài chuỗi dữ liệu
        X.append(data[i:i + seq_length]) # chuỗi con từ chỉ số i đến chỉ số i + seq_length - 1 từ data, thêm vào X và lấy một mảng con (chuỗi con) dài seq_length từ dữ liệu.
        Y.append(data[i + seq_length][feature_col]) #Lấy giá trị từ cột feature_col tại thời điểm tiếp theo ngay sau chuỗi con vừa tạo (tức là tại chỉ số i + seq_length) và thêm giá trị này vào danh sách Y.
    return np.array(X), np.array(Y) #Chuyển đổi danh sách X và Y thành các mảng NumPy và trả về chúng.

In [8]:
def train_lstm(X_train, Y_train, seq_length, epochs=100): #train_lstm với các tham số:X_train: Dữ liệu đầu vào để huấn luyện mô hình, có dạng mảng NumPy với hình dạng (số mẫu, độ dài chuỗi, số đặc trưng).
    #Y_train: Giá trị mục tiêu tương ứng với dữ liệu đầu vào.
    #seq_length: Độ dài của chuỗi đầu vào.
# epochs: Số lượng epochs (mặc định là 100) để huấn luyện mô hình
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2])) #Đảm bảo rằng dữ liệu đầu vào X_train có hình dạng (số mẫu, độ dài chuỗi, số đặc trưng) phù hợp với yêu cầu đầu vào của mô hình LSTM.
    model = Sequential() #Khởi tạo mô hình Sequential từ thư viện Keras.
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X_train.shape[2])))#Thêm lớp LSTM đầu tiên với 50 đơn vị và return_sequences=True, #return_sequences=True cho phép lớp LSTM trả về toàn bộ chuỗi đầu ra (để có thể tiếp tục với lớp LSTM tiếp theo).,input_shape=(seq_length, X_train.shape[2]) xác định hình dạng đầu vào của dữ liệu, với seq_length là độ dài chuỗi và X_train.shape[2] là số đặc trưng.
    model.add(Dropout(0.2))#Thêm lớp Dropout với tỷ lệ 0.2 (20%) để giảm overfitting bằng cách ngẫu nhiên loại bỏ 20% các đơn vị trong lớp LSTM trước đó trong mỗi lần huấn luyện.
    model.add(LSTM(50, return_sequences=False))#Thêm lớp LSTM thứ hai với 50 đơn vị (neuron)., lớp ccuoios nên là trả về đầu ra cuối cùng
    model.add(Dropout(0.2))#thêm lớp Dropout thứ hai với tỷ lệ 0.2 để giảm overfitting.
    model.add(Dense(1))#Thêm lớp Dense với 1 đơn vị đầu ra. Đây là lớp đầu ra của mô hình, dự đoán một giá trị duy nhất (hoặc nhiều giá trị nếu units được điều chỉnh).
    model.compile(optimizer='adam', loss='mean_squared_error')# Sử dụng Adam optimizer, một optimizer phổ biến cho việc huấn luyện mô hình học sâu, #Sử dụng hàm mất mát là lỗi bình phương trung bình (MSE), thường dùng cho các bài toán hồi quy
    epoch_range = tqdm(range(epochs), desc="Epochs")#Khởi tạo thanh tiến trình (tqdm) để theo dõi tiến độ của các epoch trong quá trình huấn luyện. desc="Epochs" đặt tiêu đề cho thanh tiến trình.
    for epoch in epoch_range:#Lặp qua các epoch để huấn luyện mô hình
        model.fit(X_train, Y_train, epochs=4, batch_size=32, verbose=0)# fit huấn luyện vào mô hình, epochs=4 chưa là một cái mà em mong muốn nhưng mà nếu lớn hơn nữa thì tài nguyên không đủ để thực hiện
        epoch_range.set_postfix({'loss': model.history.history['loss'][0]})#Cập nhật thanh tiến trình với giá trị hàm mất mát (loss) sau mỗi epoch.

    return model

In [9]:
 #đánh giá hiệu suất của mô hình LSTM
def evaluate_performance(model, test_data, scaler, seq_length):
    X_test, Y_test = create_sequences(test_data, seq_length, features.index('Đóng cửa'))## Tạo các chuỗi dữ liệu cho tập kiểm tra
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], len(features)))#Reshape X_test để phù hợp với yêu cầu đầu vào của mô hình LSTM

    test_predict = model.predict(X_test) # Dự đoán dữ liệu kiểm tra bằng mô hình
    
    test_predict_transformed = np.zeros((test_predict.shape[0], len(features)))# Tạo mảng trống để lưu dự đoán đã được chuyển đổi ngược
    test_predict_transformed[:, features.index('Đóng cửa')] = test_predict.flatten()   # Đặt các giá trị dự đoán vào cột tương ứng
    test_predict_transformed = scaler.inverse_transform(test_predict_transformed)[:, features.index('Đóng cửa')]# Chuyển đổi ngược dự đoán về giá trị gốc

    Y_test_transformed = np.zeros((Y_test.shape[0], len(features)))# Tạo mảng trống để lưu giá trị thực tế đã được chuyển đổi ngược
    Y_test_transformed[:, features.index('Đóng cửa')] = Y_test.flatten() # Đặt các giá trị thực tế vào cột tương ứng
    Y_test_transformed = scaler.inverse_transform(Y_test_transformed)[:, features.index('Đóng cửa')] # Chuyển đổi ngược giá trị thực tế về giá trị gốc
   # Tính toán các chỉ số đánh giá hiệu suất
    rmse = np.sqrt(mean_squared_error(Y_test_transformed, test_predict_transformed))#Root Mean Squared Error (RMSE) càng thấp càng tốt
    mae = mean_absolute_error(Y_test_transformed, test_predict_transformed)#Mean Absolute Error (MAE) càng thâp càng tốt 
    mape = np.mean(np.abs((Y_test_transformed - test_predict_transformed) / Y_test_transformed)) * 100 #Mean Absolute Percentage Error (MAPE) 
    r2 = r2_score(Y_test_transformed, test_predict_transformed)#R-squared

    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"R-squared: {r2}")


In [10]:
# Dự đoán gia đóng cửa tiếp theo
def predict_next_close(stock_data, seq_length, model, features, scaler):
    last_sequence = stock_data[features].values[-seq_length:] # Lấy chuỗi dữ liệu cuối cùng từ dữ liệu cổ phiếu, dựa trên seq_length
    last_sequence = scaler.transform(last_sequence)# Chuyển đổi dữ liệu cuối cùng về phạm vi chuẩn hóa bằng scaler
    last_sequence = np.expand_dims(last_sequence[:-1], axis=0) # Mở rộng chiều của chuỗi dữ liệu cuối cùng để phù hợp với đầu vào của mô hình LSTM
    predicted_price = model.predict(last_sequence)  # Dự đoán giá đóng cửa tiếp theo bằng mô hình LSTM
    predicted_price = np.concatenate([predicted_price, np.zeros((predicted_price.shape[0], len(features)-1))], axis=1)# Tạo mảng trống với cùng số hàng như dự đoán, nhưng với số cột bằng số đặc trưng
    predicted_price = scaler.inverse_transform(predicted_price)# Chuyển đổi ngược dự đoán về phạm vi gốc bằng scaler
    return predicted_price[0][features.index('Đóng cửa')]   # Trả về giá đóng cửa dự đoán tiếp theo

In [ ]:
models = {} #Khởi tạo một từ điển để lưu các mô hình LSTM đã huấn luyện, với mã cổ phiếu là khóa và mô hình là giá trị.
scalers = {} #Khởi tạo một từ điển để lưu các scaler đã huấn luyện, với mã cổ phiếu là khóa và scaler là giá trị.
seq_length = 20 #Đặt độ dài chuỗi đầu vào cho mô hình LSTM là 20. Điều này có nghĩa là mô hình sẽ sử dụng 20 giá trị trước đó để dự đoán giá tiếp theo.
epochs = 100 #Đặt số epoch (số lần huấn luyện qua toàn bộ tập dữ liệu) là 100 cho quá trình huấn luyện mô hình LSTM.

for symbol in data['Mã CK'].unique():
    stock_data = data[data['Mã CK'] == symbol].copy()
    stock_data_scaled, scaler = prepare_data(stock_data, features)
#Kiểm tra xem dữ liệu cổ phiếu có đủ số lượng mẫu để tạo chuỗi dữ liệu với độ dài seq_length không
    if len(stock_data_scaled) < seq_length:
        print(f"Skipping {symbol} due to insufficient data")
        continue

    tscv = TimeSeriesSplit(n_splits=5) # cross validation
    for train_index, test_index in tscv.split(stock_data_scaled):
        train_data, test_data = stock_data_scaled.iloc[train_index], stock_data_scaled.iloc[test_index] #Tách dữ liệu thành tập huấn luyện và tập kiểm tra dựa trên các chỉ số được cung cấp.

        if len(train_data) < seq_length or len(test_data) < seq_length:
            print(f"Skipping split for {symbol} due to insufficient sequence data")
            continue

        X_train, Y_train = create_sequences(train_data[features].values, seq_length, features.index('Đóng cửa'))
        X_test, Y_test = create_sequences(test_data[features].values, seq_length, features.index('Đóng cửa'))

        lstm_model = train_lstm(X_train, Y_train, seq_length, epochs) #Huấn luyện mô hình LSTM trên dữ liệu huấn luyện và số epoch được chỉ định, trả về mô hình đã huấn luyện.
        evaluate_performance(lstm_model, test_data[features].values, scaler, seq_length) #Đánh giá hiệu suất của mô hình trên dữ liệu kiểm tra và in các chỉ số đánh giá.

        models[symbol] = lstm_model
        scalers[symbol] = scaler

# Lưu mô hình và scaler
with open('models.pkl', 'wb') as file:
    pickle.dump(models, file)

with open('scalers.pkl', 'wb') as file:
    pickle.dump(scalers, file)

C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Epochs: 100%|██████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.56it/s, loss=0.00244]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/step


C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


RMSE: 1.830319450406894
MAE: 1.8094117712229498
MAPE: 16.278222491919454
R-squared: -35.15833017310082


Epochs: 100%|██████████████████████████████████████████████████████████| 100/100 [00:36<00:00,  2.74it/s, loss=0.00268]


...

Epochs: 100%|██████████████████████████████████████████████████████████| 100/100 [00:48<00:00,  2.07it/s, loss=0.00329]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step


C:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


RMSE: 0.32494410679934765
MAE: 0.24026802115142337
MAPE: 1.20825065372907
R-squared: 0.6363713386358127


Epochs:  86%|██████████████████████████████████████████████████▋        | 86/100 [00:49<00:07,  1.87it/s, loss=0.00209]